![DoctorBert](https://snag.gy/oF84Gk.jpg)


![DoctorBert](https://i.ytimg.com/vi/nPemP-Q0Xn8/hqdefault.jpg)

This is a Colab Demo of our DocProduct Tensorflow 2.0 Hackathon Project

Project details can be seen on our Github repo
https://github.com/Santosh-Gupta/DocProduct

and our Devpost page
https://devpost.com/software/nlp-doctor

In order to fit the computational limits of Google Colab, we had to reduce the amount of searchable Q&A datapoints from ~700,000 to ~200,000.
We also converted our search embeddings from float32 to float16, although this does not seem to dimish the quality of search (*but that's a whole other product to explore*)

Instructions:

-Run the first cell to install TF2.0 and FAISS

-Run the second cell to download the models and data

-Run the third cell to load the weights and data

-Input your question and search parameters into the fourth cell to play around with our model results



In [1]:
#@title Install Faiss and TF 2.0. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
!pip install https://github.com/Santosh-Gupta/MedicalQA/archive/master.zip


--2019-05-07 04:58:12--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20190507T045812Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=AgoJb3JpZ2luX2VjECkaCXVzLWVhc3QtMSJIMEYCIQCfagVxzCK8GyvTXx%2FhZRF7JyuMHqXXOBPIKTOFjIfYVwIhANr3k0ef7DcbZonD8mOG%2FhgjQ2kQqkwyloVsJmtoJQOIKtoDCCEQABoMNDU1ODY0MDk4Mzc4IgyXqbz%2FfDxJFxIIDmQqtwNBrAtGb

In [0]:
#@title Downaload all model checkpoints, and question/answer data. Double click to see this code

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os

if not os.path.exists('models'):
    os.mkdir('models')
    
if not os.path.exists('models/bertffn_crossentropy'):
    os.mkdir('models/bertffn_crossentropy')
    
if not os.path.exists('models/bertffn_crossentropy/bertffn'):
    os.mkdir('models/bertffn_crossentropy/bertffn')
    
import requests

if not os.path.exists('qa_embeddings'):
    os.mkdir('qa_embeddings')
    
if not os.path.exists('chkpoint'):
    os.mkdir('chkpoint')
    
file_id = '1thX75_cMkly5btgxTydgV6YDKnrUczZs'

download_file_from_google_drive(file_id, 'models/bertffn_crossentropy/bertffn/bertffn.data-00000-of-00001')

file_id = '11q29T38EysVPueD1PMQzZoKs5QFr3jRT'

download_file_from_google_drive(file_id, 'models/bertffn_crossentropy/bertffn/bertffn.index')

file_id = '1iI5Aow_7pQSmvpxGnMVKw9OJV-nckkxz'

download_file_from_google_drive(file_id, 'models/bertffn_crossentropy/bertffn/bertffn.checkpoint')

os.listdir('models/bertffn_crossentropy/bertffn')

import urllib.request
...
# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

os.listdir('BioBertFolder/pubmed_pmc_470k')

if not os.path.exists('FFNNFolder'):
    os.makedirs('FFNNFolder')
    
file_id = '1-258Wlpp5UwwqCPezy6DRYBozO4wBUBw'

download_file_from_google_drive(file_id, 'FFNNFolder/data-00000-of-00001')

file_id = '1-9JmtJJ_XGV0wClZxs2Px4hDL-Pi9YA_'

download_file_from_google_drive(file_id, 'FFNNFolder/index')

file_id = '1-5hczUjQfCTpBg1HhpLrgXFTkve1xuRM'

download_file_from_google_drive(file_id, 'FFNNFolder/checkpoint')

#download data

# file_id = '1blkZdV1BNWesD0mJ6Pm1H1IhCMYThkWJ'

# download_file_from_google_drive(file_id, 'Float16Embeddings.pkl')

file_id = '1jnRRljMv752su76j6aLWIyHD5oo6itMN'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded.pkl')

In [3]:
#@title Load model weights and Q&A data. Double click to see code

from Scripts.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/pubmed_pmc_470k/'
ffn_weight_file = None
bert_ffn_weight_file = 'models/bertffn_crossentropy/bertffn/bertffn'
embedding_file = 'Float16EmbeddingsExpanded.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


In [14]:
#@title Type in your question (512 word limit) and search search parameters

question_text = "Hey Dr's and Medical Professionals of Reddit. I'm in need of some assistance regarding my tonsillitis. I'm 19 turning 20 of Hispanic ethnicity,5'1 and 170 lbs. About two days ago I started getting a sore throat and I've had my fair share of sore throats so I wasn't too concerned well yesterday I woke up with severely enlarged tonsils so I started my salt gargles like any person would do, well that didn't work and my tonsils continued to become swollen to the point that my voice has become muffled and I can't pronounce any \"ey\" sounding words without sounding like a fucking idiot. Anyway, at around 6am today I got my ass to the ER and got aN attending physician he gave me a prescription of Amoxicillin 500mg and sent me on my way. Well because I slept the entire day after being kept up with my tonsil pain shit went from bad to mother fucking worse. At around 7:00PM today I felt like I had some phlegm stuck in my throat so I went to the bathroom to cough it up and it was a bloodclot. As soon as that bloodclot came out I started coughing up more of it before it subsided a lil. I started to do some salt gargles again and that \"suprisingly\" didn't work and about 10 minutes ago I coughed up about 1/3 cup of blood and as I'm writing this coughed up some more. Please help me lol."  #@param {type:"string"}

search_similarity_by = 'answer'  #@param ['answer', "question"]

number_results_toReturn=10 #@param {type:"number"}

answer_only=True #@param ["False", "True"] {type:"raw"}

returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=search_similarity_by)

print('')
for jk in range(len(returned_results)):
    print("Result ", jk+1)
    print(returned_results[jk])
    print('')


1it [00:00,  4.50it/s]


Result  1
Complete full course of Abx and if still having problems see ENT. If fever go to ED. ,How are you doing? 

Result  2
How long did the physician prescribe the amoxicillin? Did he or she do a strep test or diagnose it based on symtpoms? How long were you symptomatic beforehand?,Just take your antibiotics as directed and it should clear up. If it hasn't improved at all by the end of the course, or if you significantly worsen over the next several days, contact your doctor.

Result  3
It could very well be an abscess and urgent care won’t touch it. Go to an ER or an ENT.,Can you take a picture?
,Amoxicillin is not ototoxic, but if one tonsil is "way bigger," you need to make sure it goes down. I would go ahead and book an ENT appointment. If it goes back down to normal, then cancel it. But if it doesn't, get their opinion.

Result  4
You should see a doctor if you think you have tonsillitis

There's not much point in only taking a single dose of Amoxicillin.  You need 10 days of